In [1]:
import QUANTAXIS as QA
import datetime
import warnings
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')
import quantaxis_ext
from talib import MA_Type
import talib
import jupyter_helper
import calculator
import os
from tqdm import tqdm_notebook
import simulate
from pyecharts import Line
#加载 seaborn，并且设置默认使用 seaborn
import seaborn as sns
sns.set()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
%config InlineBackend.figure_format = 'svg'

In [2]:
zs_code = jupyter_helper.get_zs_code()
'指数代码:{0}'.format(zs_code)

start_date, end_date = jupyter_helper.get_start_end_date()
start_str = QA.QAUtil.QADate.QA_util_datetime_to_strdate(start_date)
end_str = QA.QAUtil.QADate.QA_util_datetime_to_strdate(end_date)
'开始日期:{0}'.format(start_date)
'结束日期:{0}'.format(end_date)

'指数代码:399300'

'开始日期:2005-04-08 00:00:00'

'结束日期:2019-01-18 00:00:00'

### 读取股票列表

In [3]:
lst = QA.QA_fetch_stock_list_adv()

### 计算单支股票年度数据

In [4]:
calculator.calc_perf_stats_by_year('000002',zs_code,start_date,end_date).describe()

,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,Kurtosis,Tail ratio,Daily value at risk,Alpha,Beta
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000
mean,0.477037,0.422746,0.432554,0.721778,2.165157,0.566850,-0.315396,1.160945,1.299566,0.372852,1.692717,1.310854,-0.053254,0.164177,1.148506
std,0.933083,0.840749,0.117303,1.283961,3.384774,0.264663,0.149885,0.253034,2.137131,0.435786,1.277882,0.264385,0.014831,0.217837,0.222477
min,-0.657659,-0.644298,0.267230,-1.281374,-0.895960,0.039828,-0.734027,0.819245,-1.720856,-0.233069,-0.278274,0.943801,-0.086944,-0.151970,0.818892
25%,-0.206744,-0.197516,0.360553,-0.461926,-0.594402,0.381884,-0.359899,0.925666,-0.649652,0.099297,0.939766,1.123768,-0.058208,-0.003870,0.989043
50%,0.339183,0.271671,0.403349,1.071205,1.396630,0.581602,-0.269432,1.204244,1.638998,0.170845,1.415623,1.284285,-0.050449,0.188578,1.117788
75%,0.737079,0.675503,0.485525,1.395731,3.682300,0.792359,-0.214299,1.309229,2.520587,0.549192,2.398714,1.458530,-0.044741,0.304519,1.235658
max,2.711342,2.397115,0.663328,3.092249,10.572720,0.870690,-0.174471,1.679638,5.713021,1.414937,4.838696,1.815147,-0.034153,0.592701,1.685432


### 读取所有股票的数据，写入字典

首先会尝试读取 datas 目录下的 stock_stas_year.h5 文件。如果文件不存在或有部分内容不存在，则读取部分内容。

部分内容以单支股票为单位。加入只是有部分年份不存在，又或者上次生成文件时2018年，但现在时2020年，缺少2019年的数据时，也不会重新生成。

In [14]:
def get_year_stats(start,end,lst=None):
    '''
    读取指定股票的年度分析数据。
    
    首先会尝试读取 datas 目录下的 stock_stas_year.h5 文件。如果文件不存在或有部分内容不存在，则读取部分内容。

    部分内容以单支股票为单位。加入只是有部分年份不存在，又或者上次生成文件时2018年，但现在时2020年，缺少2019年的数据时，也不会重新生成。
    
    Args:
        lst: 股票代码列表。为None时，自动获取所有列表。
    '''
    filename = os.path.join(os.getcwd(), 'datas', 'stock_stats_year.h5')
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    if lst is None or len(lst)==0:
        lst = QA.QA_fetch_stock_list_adv().code.values

    stats_by_year = pd.DataFrame()
    if os.path.exists(filename):
        stats_by_year = pd.read_hdf(filename, key='year', mode='r').astype('float64')
    has_new = False
    for stock in tqdm_notebook(lst):
        if stats_by_year.empty:
            stats_by_year = calculator.calc_perf_stats_by_year(stock, zs_code,start, end)
            has_new=True
        elif stock not in stats_by_year.index.unique(0).values:
            d = calculator.calc_perf_stats_by_year(stock, zs_code,start, end)
            if d.empty:
                continue
            stats_by_year = stats_by_year.append(d)
            has_new=True
    if has_new:
        stats_by_year.to_hdf(filename, key='year', mode='w')
        print('{0} updated.'.format(filename))
    return stats_by_year

lst_year_stats=get_year_stats(start_str,end_str,lst.code.values)

QA Error QA_fetch_stock_day_adv parameter code=002946 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=002947 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300755 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300758 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300759 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=600928 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=601298 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=601615 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None


### 所有上市日期在指数前的列表

In [15]:
valid_lst=[]
for code in set(lst_year_stats.index.get_level_values(0)):
    if lst_year_stats.loc[code].dropna().shape[0]==end_date.year-start_date.year:
        valid_lst.append(code)

### 所有上市日期在指数之前的年度数据的describe

In [16]:
year_stats_describe=lst_year_stats.loc[valid_lst].groupby('code').agg(['max','min','mean','std'])
year_stats_describe

Annual return                               Cumulative returns  \
                 max       min      mean       std                max   
code                                                                    
000001      2.659417 -0.688486  0.408981  0.953211           2.009213   
000002      2.711342 -0.657659  0.477037  0.933083           2.397115   
000004      2.556783 -0.672525  0.396515  0.989275           1.925430   
000005      2.980093 -0.668888  0.523692  1.300084           1.878543   
000006      2.199796 -0.647052  0.357953  0.774049           1.864279   
000007      2.640896 -0.652366  0.441386  0.976818           2.186441   
000008      3.117569 -0.661076  0.646649  1.331495           1.983173   
000009      3.083786 -0.763012  0.404077  1.061987           2.776722   
000011      4.480859 -0.745120  0.387136  1.295423           3.183801   
000012      3.575354 -0.620556  0.393158  1.099445           2.243717   
000014      2.738584 -0.499991  0.419594  0.906322           1.646747   
000016      2.219128 -0.467921  0.325312  0.825285           1.389126   
000018      3.940121 -0.681828  0.444416  1.195570           1.705263   
000019     12.744131 -0.775666  1.031809  3.421839           2.945996   
000020      4.627796 -0.777028  0.498289  1.643169           3.443548   
000021      2.220253 -0.771868  0.281628  0.773495           2.017685   
000023      2.167031 -0.680829  0.460617  1.013535           1.505172   
000025      8.396656 -0.770661  1.013580  2.598211           7.223529   
000026      3.635335 -0.724910  0.402169  1.108903           2.256693   
000027      2.155013 -0.673886  0.239004  0.739273           1.946447   
000028      2.172552 -0.344147  0.382711  0.753045           1.975450   
000031      3.256038 -0.685956  0.441546  1.151276           2.460569   
000032      1.918545 -0.604494  0.203750  0.719978           1.749944   
000034     18.583346 -0.864746  1.815721  5.103833           3.656085   
000036      3.842951 -0.780117  0.474851  1.272269           2.586022   
000037      4.270574 -0.666712  0.350215  1.272355           1.200608   
000039      2.022129 -0.768062  0.249920  0.781498           1.394941   
000040      2.289779 -0.634149  0.377713  0.959599           2.027119   
000042      3.011625 -0.764734  0.459914  1.136519           2.725913   
000043      2.514705 -0.799032  0.416052  0.932992           1.981284   
...              ...       ...       ...       ...                ...   
600960      1.600363 -0.667179  0.228076  0.692479           1.503597   
600961      2.709282 -0.873163  0.458311  1.180563           2.413064   
600962      2.565904 -0.693814  0.397358  1.001288           2.256334   
600963      2.184270 -0.759094  0.348884  0.916457           1.944941   
600965      2.117410 -0.549767  0.309720  0.742848           1.953112   
600966      3.142720 -0.676467  0.309541  0.945010           2.721724   
600967      1.992153 -0.634749  0.363473  0.633006           1.754839   
600969      2.233421 -0.664952  0.264141  0.803723           2.043478   
600970      1.804190 -0.564545  0.309506  0.774546           1.313598   
600971      3.703796 -0.815886  0.428899  1.298893           3.342688   
600973      2.034407 -0.810185  0.396855  0.857589           1.878180   
600975      3.739090 -0.621370  0.360859  1.099796           3.346666   
600976      2.274106 -0.552667  0.286218  0.704542           2.079785   
600978     31.073207 -0.786242  2.509713  8.272540           2.596154   
600979      1.395196 -0.527487  0.201471  0.529237           1.281742   
600980      1.777163 -0.571190  0.200415  0.678814           1.634615   
600981      6.259888 -0.731365  0.611130  1.806881           5.502801   
600982      1.882665 -0.629687  0.215015  0.682313           1.695262   
600983      2.260375 -0.450425  0.350286  0.901837           2.081954   
600984      3.872343 -0.806857  0.448274  1.293371           3.462006   
600985      2.055779 -0.534994  0.257040  0.747355           1.92

#### 按照年度夏普比率均值倒序排序

In [17]:
ysd_sr=year_stats_describe['Sharpe ratio']

In [18]:
ysd_sr.sort_values('mean',ascending=False)[0:5]

,max,min,mean,std
code,,,,
000605,17.338131,-2.131381,1.456605,4.709920
600745,14.403512,-1.922585,1.281992,4.072452
600519,4.781666,-1.447703,1.127921,1.684034
000895,9.936704,-1.023082,1.122892,2.795773
000651,3.006778,-0.735689,1.106123,1.133413


#### 按照年度夏普比率标准差顺序排序

In [19]:
ysd_sr.sort_values('std')[0:5]

,max,min,mean,std
code,,,,
600377,1.698381,-1.140232,0.526205,0.786980
600885,1.932153,-0.559473,0.583153,0.793969
600846,1.850436,-1.065475,0.426364,0.795867
600636,1.834666,-1.006742,0.422275,0.811862
000955,1.586920,-0.599248,0.355083,0.819009


#### 从年度数据的describe中按照std从小到大，mean从大到小选出10支股票

In [20]:
def _sort_buy_std_and_mean(ysd_sr):
    '''根据年度数据的describe中的std和mean综合排序
    std按照从小到大，
    mean按照从大到小
    '''
    std_df=pd.DataFrame(ysd_sr.sort_values('std').reset_index()['code'])
    std_df['order']=std_df.index
    std_df=std_df.set_index('code')
    mean_df=pd.DataFrame(ysd_sr.sort_values('mean',ascending=False).reset_index()['code'])
    mean_df['order']=mean_df.index
    mean_df=mean_df.set_index('code')
    df=std_df.join(mean_df,lsuffix='_std',rsuffix='_mean')
    df['score']=df['order_std']+df['order_mean']
    return df
# _sort_buy_std_and_mean(ysd_sr).sort_values('score')[0:10]
# ysd_sr.loc[_sort_buy_std_and_mean(ysd_sr).sort_values('score')[0:10].index.values]
_sort_buy_std_and_mean(ysd_sr).sort_values('score')[0:10].index.values

array(['600436', '000538', '600967', '600867', '600612', '600588',
       '600801', '000963', '000998', '600406'], dtype=object)

### 模拟选出的10支股票，从开始到结束日期的表现

每支股票初始资金10000

In [79]:
class QA_Performance_Ext(QA.QA_Performance):
    '''此方法包含在finance_ai_py\quantaxis_ext.py'''

    @property
    def pnl_fifo_qfq(self):
        pnl = super(QA_Performance_Ext, self).pnl_fifo
        #         pnl['buy_price_qfq'] = pnl['buy_price']
        pnl = pnl.reset_index()
        for idx in pnl.index:
            s = QA.QAUtil.QADate.QA_util_datetime_to_strdate(
                pnl.loc[idx, 'buy_date'])
            e = QA.QAUtil.QADate.QA_util_datetime_to_strdate(
                pnl.loc[idx, 'sell_date'])
            c = pnl.loc[idx, 'code']
            new_value = QA.QA_fetch_stock_day_adv(c, s, e).to_qfq()
            pnl.loc[idx, 'buy_price_qfq'] = np.round(
                new_value.select_day(s).close[0], 4)
        pnl = pnl.assign(
            pnl_money_qfq=(pnl.sell_price - pnl.buy_price_qfq) * pnl.amount)
        return pnl.set_index('code')


def simulate_bbands_report(codes):
    '''返回报告DataFrame和{code:QA_Account}的字典'''
    df = simulate.bbands(codes)
    income = []  #总收益
    cost = []  #总投入
    winner = []  #胜率
    days_min = []
    days_max=[]
    days_mean=[]
    mean = []#收益均值
    times = []#交易次数（买卖算一次）
    pnl_money_qfq_sum = []
    for code in codes:
        account = df[code]
        Performance_ext = QA_Performance_Ext(account)
        qfq=Performance_ext.pnl_fifo_qfq
        income.append(
            np.round(qfq.pnl_money_qfq.sum(), 2))
        cost.append(
            np.round((qfq.buy_price_qfq *
                      qfq.amount).sum(), 2))
        days_min.append((qfq.sell_date-qfq.buy_date).min().days)
        days_max.append((qfq.sell_date-qfq.buy_date).max().days)
        days_mean.append((qfq.sell_date-qfq.buy_date).mean().days)
        mean.append(
            np.round(qfq.pnl_money_qfq.mean(), 2))
        times.append(qfq.shape[0])
    report = pd.DataFrame.from_dict(
        {
            'income': income,
            'cost': cost,
            #         'win_rate': winner,
            'mean': mean,
            'times': times,
            'days_min': days_min,
            'days_max': days_max,
            'days_mean': days_mean
        },
        orient='index',
        columns=codes)
    report=report.T
    report['rate'] = report['income'] / report['cost']
    return report,df

#### 前10名

In [90]:
_sort_buy_std_and_mean(ysd_sr).sort_values('score')[0:10].index.values
lst.loc[_sort_buy_std_and_mean(ysd_sr).sort_values('score')[0:10].index.values]

array(['600436', '000538', '600967', '600867', '600612', '600588',
       '600801', '000963', '000998', '600406', '600885', '600572',
       '002025', '600498', '000860', '002013', '600116', '600332',
       '002001', '000789', '600763', '000739', '600486', '600323',
       '002038', '000977', '600377', '600535', '600566', '600897'],
      dtype=object)

,code,decimal_point,name,pre_close,sec,sse,volunit
code,,,,,,,
600436,600436,2,片仔癀,86.000000,stock_cn,sh,100
000538,000538,2,云南白药,75.849998,stock_cn,sz,100
600967,600967,2,内蒙一机,760.012500,stock_cn,sh,100
600867,600867,2,通化东宝,1224.040000,stock_cn,sh,100
600612,600612,2,老凤祥,212.200001,stock_cn,sh,100
600588,600588,2,用友网络,24.230000,stock_cn,sh,100
600801,600801,2,华新水泥,17.430000,stock_cn,sh,100
000963,000963,2,华东医药,26.740000,stock_cn,sz,100
000998,000998,2,隆平高科,1384.025000,stock_cn,sz,100


In [80]:
report,account_dict=simulate_bbands_report(_sort_buy_std_and_mean(ysd_sr).sort_values('score')[0:30].index.values)

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

In [94]:
report[:10]

,times,income,days_max,days_min,cost,mean,days_mean,rate
600436,10.0,7022.04,291.0,36.0,46010.96,702.20,163.0,0.152617
000538,8.0,4367.16,165.0,47.0,41613.84,545.90,99.0,0.104945
600967,12.0,2254.74,417.0,28.0,15101.26,187.90,196.0,0.149308
600867,17.0,4166.35,412.0,39.0,18849.65,245.08,160.0,0.221031
600612,9.0,4073.84,469.0,69.0,22150.16,452.65,250.0,0.183919
600588,12.0,1209.49,513.0,39.0,22783.51,100.79,175.0,0.053086
600801,8.0,-237.52,293.0,35.0,9634.52,-29.69,127.0,-0.024653
000963,11.0,1697.43,282.0,41.0,26229.57,154.31,135.0,0.064714
000998,13.0,3474.00,181.0,9.0,25215.00,267.23,112.0,0.137775
600406,11.0,3351.38,769.0,36.0,15780.62,304.67,263.0,0.212373


#### 后10名

In [95]:
report_last_10,account_dict_last_10=simulate_bbands_report(_sort_buy_std_and_mean(ysd_sr).sort_values('score')[-10:].index.values)

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

In [96]:
report_last_10

,times,income,days_max,days_min,cost,mean,days_mean,rate
000633,17.0,-826.00,336.0,31.0,11913.00,-48.59,164.0,-0.069336
000959,16.0,-579.11,390.0,27.0,7286.11,-36.19,150.0,-0.079481
000628,23.0,513.00,513.0,13.0,19628.00,22.30,148.0,0.026136
600654,15.0,-434.91,468.0,38.0,7236.91,-28.99,208.0,-0.060096
000155,7.0,201.00,592.0,55.0,4049.00,28.71,244.0,0.049642
600146,8.0,1594.00,786.0,49.0,13065.00,199.25,254.0,0.122005
600399,13.0,-396.92,436.0,63.0,8305.92,-30.53,175.0,-0.047788
600844,13.0,1367.00,371.0,32.0,12033.00,105.15,172.0,0.113604
600734,16.0,1847.00,531.0,25.0,7647.00,115.44,183.0,0.241533
600248,12.0,1035.80,296.0,3.0,7629.20,86.32,136.0,0.135768


#### 前10名和后10名的对比

In [97]:
report[:10].rate.mean(),report_last_10.rate.mean()
report[:10].income.sum(),report_last_10.income.sum()
report[:10].cost.sum(),report_last_10.cost.sum()

(0.12551153219978284, 0.04319870166113081)

(31378.91, 4320.86)

(243369.08999999997, 98793.14)

In [106]:
account_dict['600436'].message

{'account_cookie': 'Acc_UKQ6gNFh',
 'allow_sellopen': False,
 'allow_t0': False,
 'broker': 'backtest',
 'cash': [10000,
  6446.0,
  3323.0,
  9763.548,
  7775.548000000001,
  9495.821,
  7158.821,
  9611.361,
  5930.361000000001,
  2405.3610000000003,
  13429.321,
  7402.321,
  14462.249,
  8028.248999999999,
  18671.59,
  8164.59,
  16035.706,
  10829.706,
  16557.966999999997,
  6224.966999999997],
 'commission_coeff': 0.00025,
 'current_time': 'None',
 'end_date': '2018-08-07',
 'history': [['2007-09-13 00:00:00',
   '600436',
   35.49,
   100,
   6446.0,
   'Order_GJxlmVWB',
   'Order_GJxlmVWB',
   'Trade_MF8TVvsB',
   'Acc_UKQ6gNFh',
   5,
   0,
   None,
   0],
  ['2007-10-30 00:00:00',
   '600436',
   31.18,
   100,
   3323.0,
   'Order_JDfaoxzm',
   'Order_JDfaoxzm',
   'Trade_2bzVRutO',
   'Acc_UKQ6gNFh',
   5,
   0,
   None,
   0],
  ['2008-05-13 00:00:00',
   '600436',
   32.26,
   -200,
   9763.548,
   'Order_DPKIbjCS',
   'Order_DPKIbjCS',
   'Trade_JceNVPzi',
   'Acc_UKQ6